In [23]:
import tensorflow as tf
import sys
import string
import collections
import numpy as np
import time

In [6]:
# Split text into words
python3 = sys.version_info[0] == 3
with open('data/lovecraft/lovecraft.txt', 'r') as f:
    input_str = f.read().lower()
    if python3:
        trans = input_str.maketrans('', '', string.punctuation)
        input_str = input_str.translate(trans)
    else:
        input_str = input_str.decode('utf-8').translate(None, string.punctuation)
    words = input_str.split()
    num_words = len(words)
print(num_words)

499120


In [10]:
# Convert words to values
word_freq = collections.Counter(words).most_common()
vocab_size = len(word_freq)
print(vocab_size)

29807


In [11]:
lookup = dict()
for word, _ in word_freq:
    lookup[word] = len(lookup)

In [17]:
# print(lookup)

In [16]:
input_vals = np.asarray([[lookup[str(word)]] for word in words])
print(input_vals)
input_vals = input_vals.reshape(-1,)
print(input_vals)

[[    0]
 [  293]
 [  128]
 ...
 [29804]
 [29805]
 [29806]]
[    0   293   128 ... 29804 29805 29806]


In [18]:
# Set values
input_size = 6
batch_size = 10
num_hidden = 600
# Placeholders
input_holder = tf.placeholder(tf.float32, [batch_size, input_size])
label_holder = tf.placeholder(tf.float32, [batch_size, vocab_size])

In [19]:
# Reshape input and feed to RNN
cell = tf.nn.rnn_cell.BasicRNNCell(num_hidden)
outputs, _ = tf.nn.static_rnn(cell, [input_holder], dtype=tf.float32)

In [29]:
print(np.shape(outputs))

(1,)


In [30]:
# Compute loss
weights = tf.Variable(tf.random_normal([num_hidden, vocab_size]))
biases = tf.Variable(tf.random_normal([vocab_size]))
model = tf.matmul(outputs[-1], weights) + biases
print(np.shape(model))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model,
labels=label_holder))

(10, 29807)


In [21]:
# Create optimizer and check result
optimizer = tf.train.AdagradOptimizer(0.1).minimize(loss)
check = tf.equal(tf.argmax(model, 1), tf.argmax(label_holder, 1))
correct = tf.reduce_sum(tf.cast(check, tf.float32))

In [28]:
# Execute the graph
start_time = time.time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_block = np.empty([batch_size, input_size])
label_block = np.empty([batch_size, vocab_size])
step = 0
num_correct = 0.
accuracy = 0.
while accuracy < 0.95:
    for i in range(batch_size):
        offset = np.random.randint(num_words-(input_size+1))
        input_block[i, :] = input_vals[offset:offset+input_size]
        label_block[i, :] = np.eye(vocab_size)[input_vals[offset+input_size]]
    _, corr = sess.run([optimizer, correct],
    feed_dict={input_holder: input_block, label_holder: label_block})
    num_correct += corr
    if step % 10 == 0:
        accuracy = num_correct/(10*batch_size)
        print('Step', step, '- Accuracy =', accuracy)
        num_correct = 0
    step += 1

Step 0 - Accuracy = 0.0
Step 10 - Accuracy = 0.0
Step 20 - Accuracy = 0.02
Step 30 - Accuracy = 0.0
Step 40 - Accuracy = 0.0
Step 50 - Accuracy = 0.02
Step 60 - Accuracy = 0.01
Step 70 - Accuracy = 0.02
Step 80 - Accuracy = 0.01
Step 90 - Accuracy = 0.02
Step 100 - Accuracy = 0.01
Step 110 - Accuracy = 0.04
Step 120 - Accuracy = 0.0
Step 130 - Accuracy = 0.01
Step 140 - Accuracy = 0.03
Step 150 - Accuracy = 0.01
Step 160 - Accuracy = 0.02
Step 170 - Accuracy = 0.0
Step 180 - Accuracy = 0.0
Step 190 - Accuracy = 0.01
Step 200 - Accuracy = 0.01
Step 210 - Accuracy = 0.02
Step 220 - Accuracy = 0.02
Step 230 - Accuracy = 0.02
Step 240 - Accuracy = 0.01
Step 250 - Accuracy = 0.02
Step 260 - Accuracy = 0.01
Step 270 - Accuracy = 0.01
Step 280 - Accuracy = 0.01
Step 290 - Accuracy = 0.01
Step 300 - Accuracy = 0.02
Step 310 - Accuracy = 0.0
Step 320 - Accuracy = 0.04
Step 330 - Accuracy = 0.03
Step 340 - Accuracy = 0.02
Step 350 - Accuracy = 0.02
Step 360 - Accuracy = 0.0
Step 370 - Accuracy =

KeyboardInterrupt: 

In [ ]:
# Display timing result
duration = time.time() - start_time
print('Time to reach 95% accuracy: {:.2f} seconds'.format(duration))